In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import datetime as dt

from sympy import symbols, Symbol, Eq, nsolve, solve, sqrt

import math
from decimal import Decimal, getcontext
getcontext().prec = 50

data_dir = '/Users/zhicong/Dropbox/DEXs'
pool_name = 'USDC-WETH 500 10'

# Functions

In [2]:
def Get_Price(_tick_):
    return Decimal(1/(1.0001**_tick_*10**-12))


def Get_Liquidity(_token0_amount_, _token1_amount_, _price_lower_, _price_upper_):

    L = Symbol('L')
    L = solve((_token0_amount_ + L/(_price_upper_.sqrt())) * (_token1_amount_ + L*(_price_lower_.sqrt())) - L**2, L, prec = 10)
    L = L[1]

    return L

# Read and Clean Data

Liquidity provision action

In [3]:
lp_action = pd.read_csv(os.path.join(data_dir, "uniswapv3_lp_actions", pool_name + ".csv"))
lp_action.columns = [x.upper() for x in lp_action.columns.tolist()]
lp_action.dropna(inplace = True)

# Final cleaning
lp_action['BLOCK_NUMBER'] = lp_action['BLOCK_NUMBER'].astype(int)
lp_action.sort_values(by = ['BLOCK_NUMBER'], ascending = False, inplace = True)
lp_action['AMOUNT0_ADJUSTED'] = lp_action['AMOUNT0_ADJUSTED'].astype(float)
lp_action['AMOUNT1_ADJUSTED'] = lp_action['AMOUNT1_ADJUSTED'].astype(float)
lp_action['BLOCK_TIMESTAMP'] = pd.to_datetime([x[:-5] for x in lp_action['BLOCK_TIMESTAMP'].astype(str)])
lp_action = lp_action[(lp_action['AMOUNT0_ADJUSTED'] != 0) | (lp_action['AMOUNT1_ADJUSTED'] != 0)]

# Changing signs of addition and extraction
#lp_action['AMOUNT0_ADJUSTED'] = np.where(lp_action['ACTION'] == 'DECREASE_LIQUIDITY', -lp_action['AMOUNT0_ADJUSTED'], lp_action['AMOUNT0_ADJUSTED'])
#lp_action['AMOUNT0_USD'] = np.where(lp_action['ACTION'] == 'DECREASE_LIQUIDITY', -lp_action['AMOUNT0_USD'], lp_action['AMOUNT0_USD'])
#lp_action['AMOUNT1_ADJUSTED'] = np.where(lp_action['ACTION'] == 'DECREASE_LIQUIDITY', -lp_action['AMOUNT1_ADJUSTED'], lp_action['AMOUNT1_ADJUSTED'])
#lp_action['AMOUNT1_USD'] = np.where(lp_action['ACTION'] == 'DECREASE_LIQUIDITY', -lp_action['AMOUNT1_USD'], lp_action['AMOUNT1_USD'])
lp_action['ACTION'] = lp_action['ACTION'].replace({'INCREASE_LIQUIDITY':'Increase Liquidity',
                                                   'DECREASE_LIQUIDITY':'Decrease Liquidity'})
lp_action.rename(columns = {'LIQUIDITY_PROVIDER':'SENDER', 'ACTION':'EVENT_NAME'}, inplace = True)

# Get lower and upper price from tick
lp_action['PRICE_LOWER'] = lp_action['TICK_UPPER'].apply(Get_Price)
lp_action['PRICE_UPPER'] = lp_action['TICK_LOWER'].apply(Get_Price)

# Reverse to make sure LPs are sorted in the same order as they are executed
lp_action = lp_action.iloc[::-1]
lp_action.reset_index(drop = True, inplace = True)

lp_action.head()

,POOL_NAME,EVENT_NAME,BLOCK_NUMBER,BLOCK_TIMESTAMP,SENDER,AMOUNT0_ADJUSTED,TOKEN0_SYMBOL,AMOUNT0_USD,AMOUNT1_ADJUSTED,TOKEN1_SYMBOL,AMOUNT1_USD,TICK_LOWER,TICK_UPPER,PRICE_LOWER,PRICE_UPPER
0,USDC-WETH 500 10,Increase Liquidity,12376729,2021-05-05 21:42:11,0xb2ef52180d1e5f4835f4e343251286fa84743456,2995.507735,USDC,2995.292327,1.000000,WETH,3437.435736,191150,198080,2499.91359912486268513021059334278106689453125,4998.9181710649063461460173130035400390625
1,USDC-WETH 500 10,Decrease Liquidity,12376958,2021-05-05 22:27:41,0xb2ef52180d1e5f4835f4e343251286fa84743456,2588.187276,USDC,2588.001158,0.931484,WETH,3201.917248,191150,198080,2499.91359912486268513021059334278106689453125,4998.9181710649063461460173130035400390625
2,USDC-WETH 500 10,Increase Liquidity,12377035,2021-05-05 22:44:11,0x35f5a1ec10deee1256351e37bc78dc80345895f8,1.000000,USDC,0.999928,0.000146,WETH,0.500964,194990,195010,3398.18281290132699723471887409687042236328125,3404.98563895004917867481708526611328125
3,USDC-WETH 500 10,Increase Liquidity,12377266,2021-05-05 23:34:39,0x9f9b987becfe15aca93ce9208cca146f252c8d46,598.469729,USDC,598.426693,2.600000,WETH,8937.332900,186730,195460,3248.661520935142107191495597362518310546875,7777.2237085991100684623233973979949951171875
4,USDC-WETH 500 10,Increase Liquidity,12377308,2021-05-05 23:44:49,0x9f9b987becfe15aca93ce9208cca146f252c8d46,3405.610348,USDC,3405.365449,0.000000,WETH,0.000000,194660,194990,3404.98563895004917867481708526611328125,3519.21893691676132220891304314136505126953125


Swaps action

In [4]:
swaps = pd.DataFrame(columns = ['BLOCK_NUMBER','BLOCK_TIMESTAMP','TX_HASH','SENDER','TX_TO','PLATFORM','POOL_NAME','CONTRACT_ADDRESS',
                                'EVENT_NAME','AMOUNT_IN','SYMBOL_IN','AMOUNT_OUT','SYMBOL_OUT'])

for i in range(1,4):
    temp = pd.read_parquet(os.path.join(data_dir, "dex_swaps", pool_name + "_" + str(i) + ".gzip"))
    swaps = pd.concat([swaps,temp], axis = 0)
    del temp

# Final cleaning
swaps.reset_index(drop = True, inplace = True)
swaps['BLOCK_NUMBER'] = swaps['BLOCK_NUMBER'].astype(int)
swaps['AMOUNT_IN'] = swaps['AMOUNT_IN'].astype(float)
swaps['AMOUNT_OUT'] = swaps['AMOUNT_OUT'].astype(float)
swaps['BLOCK_TIMESTAMP'] = pd.to_datetime(swaps['BLOCK_TIMESTAMP'])

# Since the pool charges a transaction fee of 5 basis point on the incoming token, and stored it elsewhere (not in liquidity pool), we subtract that
#swaps['AMOUNT_IN'] = swaps['AMOUNT_IN']*(1-0.0005)

# Changing signs of addition and extraction
swaps['AMOUNT_OUT'] = -swaps['AMOUNT_OUT']
swaps['AMOUNT0_ADJUSTED'] = np.where(swaps['SYMBOL_IN'] == 'WETH', swaps['AMOUNT_OUT'], swaps['AMOUNT_IN'])
swaps['TOKEN0_SYMBOL'] = 'USDC'
swaps['AMOUNT1_ADJUSTED'] = np.where(swaps['SYMBOL_IN'] == 'WETH', swaps['AMOUNT_IN'], swaps['AMOUNT_OUT'])
swaps['TOKEN1_SYMBOL'] = 'WETH'

swaps = swaps[['POOL_NAME','BLOCK_NUMBER','BLOCK_TIMESTAMP','SENDER','EVENT_NAME','AMOUNT0_ADJUSTED','TOKEN0_SYMBOL','AMOUNT1_ADJUSTED','TOKEN1_SYMBOL']]
swaps.sort_values(by = 'BLOCK_NUMBER', ascending = True, inplace = True)
swaps.reset_index(drop = True, inplace = True)

swaps.head()

,POOL_NAME,BLOCK_NUMBER,BLOCK_TIMESTAMP,SENDER,EVENT_NAME,AMOUNT0_ADJUSTED,TOKEN0_SYMBOL,AMOUNT1_ADJUSTED,TOKEN1_SYMBOL
0,USDC-WETH 500 10,12376891,2021-05-05 22:15:01,0xe592427a0aece92de3edee1f18e0157c05861564,Swap,-119.744094,USDC,0.035000,WETH
1,USDC-WETH 500 10,12377278,2021-05-05 23:37:55,0xe592427a0aece92de3edee1f18e0157c05861564,Swap,499.756414,USDC,-0.144241,WETH
2,USDC-WETH 500 10,12377345,2021-05-05 23:51:36,0xe592427a0aece92de3edee1f18e0157c05861564,Swap,365.000000,USDC,-0.103492,WETH
3,USDC-WETH 500 10,12377364,2021-05-05 23:56:34,0xe592427a0aece92de3edee1f18e0157c05861564,Swap,-176.180828,USDC,0.050000,WETH
4,USDC-WETH 500 10,12377369,2021-05-05 23:57:36,0xe592427a0aece92de3edee1f18e0157c05861564,Swap,-514.279985,USDC,0.146000,WETH


Merging

In [5]:
# Removing new LPs
lp_action = lp_action[lp_action['BLOCK_NUMBER'] <= swaps['BLOCK_NUMBER'].max()]
lp_action.reset_index(drop = True, inplace = True)

data = pd.concat([swaps, lp_action], axis = 0)
data.sort_values(by = 'BLOCK_NUMBER', ascending = True, inplace = True)
data.rename(columns = {'AMOUNT0_ADJUSTED':'Delta USDC', 'AMOUNT1_ADJUSTED':'Delta WETH'}, inplace = True)
data.reset_index(drop = True, inplace = True)
data = data[~((data['Delta WETH'] == 0) & (data['Delta USDC'] == 0))]

data = data[['POOL_NAME','BLOCK_NUMBER','BLOCK_TIMESTAMP','SENDER','EVENT_NAME','Delta WETH','Delta USDC','TICK_LOWER','TICK_UPPER','PRICE_LOWER','PRICE_UPPER']]

data.head()

,POOL_NAME,BLOCK_NUMBER,BLOCK_TIMESTAMP,SENDER,EVENT_NAME,Delta WETH,Delta USDC,TICK_LOWER,TICK_UPPER,PRICE_LOWER,PRICE_UPPER
0,USDC-WETH 500 10,12376729,2021-05-05 21:42:11,0xb2ef52180d1e5f4835f4e343251286fa84743456,Increase Liquidity,1.000000,2995.507735,191150.0,198080.0,2499.91359912486268513021059334278106689453125,4998.9181710649063461460173130035400390625
1,USDC-WETH 500 10,12376891,2021-05-05 22:15:01,0xe592427a0aece92de3edee1f18e0157c05861564,Swap,0.035000,-119.744094,NaN,NaN,NaN,NaN
2,USDC-WETH 500 10,12376958,2021-05-05 22:27:41,0xb2ef52180d1e5f4835f4e343251286fa84743456,Decrease Liquidity,0.931484,2588.187276,191150.0,198080.0,2499.91359912486268513021059334278106689453125,4998.9181710649063461460173130035400390625
3,USDC-WETH 500 10,12377035,2021-05-05 22:44:11,0x35f5a1ec10deee1256351e37bc78dc80345895f8,Increase Liquidity,0.000146,1.000000,194990.0,195010.0,3398.18281290132699723471887409687042236328125,3404.98563895004917867481708526611328125
4,USDC-WETH 500 10,12377266,2021-05-05 23:34:39,0x9f9b987becfe15aca93ce9208cca146f252c8d46,Increase Liquidity,2.600000,598.469729,186730.0,195460.0,3248.661520935142107191495597362518310546875,7777.2237085991100684623233973979949951171875


# Descriptive Statistics

We will be building data descriptives here. One assumption we made here is that each address represents one single agent.

## Basic Descriptive

In [ ]:
print("Data period:", data['BLOCK_TIMESTAMP'].min().strftime("%d %B %Y"), "-", data['BLOCK_TIMESTAMP'].max().strftime("%d %B %Y"))

In [ ]:
print("Number of actions in our dataset:", len(data))
print("Number of liquidity provision action:", len(data[data['EVENT_NAME'].isin(['Increase Liquidity','Decrease Liquidity'])]))
print("Number of swap action:", len(data[data['EVENT_NAME'].isin(['Swap'])]))

## Liquidity Providers

In [ ]:
print("Total Number of unique Liquidity Provider:", len(lp_action['SENDER'].unique()))

In [ ]:
print(len(lp_action))

In [ ]:
lp_action['L'] = 0

for i in range(len(lp_action)):
    
    if i%1000 == 0: print(i)

    L = Get_Liquidity(lp_action.loc[i,'AMOUNT1_ADJUSTED'], lp_action.loc[i,'AMOUNT0_ADJUSTED'], lp_action.loc[i,'PRICE_LOWER'], lp_action.loc[i,'PRICE_UPPER'])
    lp_action.loc[i,'L'] = L


In [ ]:
lp_pool_share = lp_action[['SENDER','AMOUNT0_USD','AMOUNT1_USD']].copy()
lp_pool_share['USD'] = lp_pool_share['AMOUNT0_USD'] + lp_pool_share['AMOUNT1_USD']
lp_pool_share.groupby('SENDER').agg({'USD':np.sum})

# Traders

In [ ]:
print("Total Number of unique Traders:", len(swaps['SENDER'].unique()))